In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from category_encoders.one_hot import OneHotEncoder as OHE2
from sklearn.metrics import accuracy_score

from pprint import pprint
import re
import pickle

In [2]:
dataset = pd.read_csv("./data_df/dataset_df__13-features__price.csv")

In [3]:
dataset.head()

,لون السيارة,نوع الوقود,أصل السيارة,رخصة السيارة,نوع الجير,الزجاج,قوة الماتور,عداد السيارة,عدد الركاب,وسيلة الدفع,معروضة,أصحاب سابقون,إضافات,price
0,أبيض عاجي,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,2000,75000,4+1,نقدا فقط,للبيع فقط,يد اولى,\n\nمُكيّف\nإغلاق مركزي\nجهاز إنذار\nمسجل CD\n...,100000
1,سكني,ديزل,خصوصي,فلسطينية,نصف اوتوماتيك,الكتروني,2500,130000,7+1,إمكانية التقسيط,للبيع أو التبديل,2,\n\nمُكيّف\nإغلاق مركزي\nجهاز إنذار\nمسجل CD\n...,60000
2,سكني,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1600,NaN,NaN,نقدا فقط,للبيع فقط,NaN,\n\nمُكيّف\nإغلاق مركزي\nجهاز إنذار\nمسجل CD\n...,43500
3,بيج,بنزين,خصوصي,فلسطينية,عادي,يدوي,906,شغال,4+1,إمكانية التقسيط,للبيع فقط,00,\n\nمُكيّف\nمسجل CD\n\n,5500
4,فضي,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1200,38000,4+1,نقدا فقط,للبيع فقط,NaN,\n\nمُكيّف\nإغلاق مركزي\nجهاز إنذار\nجنطات مغن...,54000


In [4]:
# shape of the data
dataset.shape

(6864, 14)

In [5]:
# information about the data
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6864 entries, 0 to 6863
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   لون السيارة   6864 non-null   object
 1   نوع الوقود    6864 non-null   object
 2   أصل السيارة   6864 non-null   object
 3   رخصة السيارة  6864 non-null   object
 4   نوع الجير     6864 non-null   object
 5   الزجاج        6864 non-null   object
 6   قوة الماتور   6864 non-null   int64 
 7   عداد السيارة  4923 non-null   object
 8   عدد الركاب    6087 non-null   object
 9   وسيلة الدفع   6864 non-null   object
 10  معروضة        6864 non-null   object
 11  أصحاب سابقون  5214 non-null   object
 12  إضافات        6864 non-null   object
 13  price         6864 non-null   int64 
dtypes: int64(2), object(12)
memory usage: 750.9+ KB


In [6]:
dataset

,لون السيارة,نوع الوقود,أصل السيارة,رخصة السيارة,نوع الجير,الزجاج,قوة الماتور,عداد السيارة,عدد الركاب,وسيلة الدفع,معروضة,أصحاب سابقون,إضافات,price
0,أبيض عاجي,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,2000,75000,4+1,نقدا فقط,للبيع فقط,يد اولى,\n\nمُكيّف\nإغلاق مركزي\nجهاز إنذار\nمسجل CD\n...,100000
1,سكني,ديزل,خصوصي,فلسطينية,نصف اوتوماتيك,الكتروني,2500,130000,7+1,إمكانية التقسيط,للبيع أو التبديل,2,\n\nمُكيّف\nإغلاق مركزي\nجهاز إنذار\nمسجل CD\n...,60000
2,سكني,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1600,NaN,NaN,نقدا فقط,للبيع فقط,NaN,\n\nمُكيّف\nإغلاق مركزي\nجهاز إنذار\nمسجل CD\n...,43500
3,بيج,بنزين,خصوصي,فلسطينية,عادي,يدوي,906,شغال,4+1,إمكانية التقسيط,للبيع فقط,00,\n\nمُكيّف\nمسجل CD\n\n,5500
4,فضي,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1200,38000,4+1,نقدا فقط,للبيع فقط,NaN,\n\nمُكيّف\nإغلاق مركزي\nجهاز إنذار\nجنطات مغن...,54000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6859,فيراني,كهرباء,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1000,130000,4+1,إمكانية التقسيط,للبيع فقط,2,\n\nمُكيّف\nإغلاق مركزي\nجهاز إنذار\nجنطات مغن...,45000
6860,أبيض,ديزل,عمومي,فلسطينية,عادي,الكتروني,2400,00000,7+1,نقدا فقط,للبيع فقط,5,\n\n\n,48000
6861,فضي,ديزل,خصوصي,فلسطينية,عادي,الكتروني,1600,50000,4+1,نقدا فقط,للبيع فقط,يد صفر,\n\nمُكيّف\nإغلاق مركزي\nجهاز إنذار\nمسجل CD\n...,87000
6862,بترولي,ديزل,خصوصي,فلسطينية,اوتوماتيك,الكتروني,2200,100,6+1,نقدا فقط,للبيع فقط,ثانيه,\n\nمُكيّف\nإغلاق مركزي\nجهاز إنذار\nمسجل CD\n...,126000


In [7]:
# Observe null values proportion for each feature
df_null = dataset.isnull().mean()
df_null.sort_values(ascending=False).head()

عداد السيارة    0.282780
أصحاب سابقون    0.240385
عدد الركاب      0.113199
لون السيارة     0.000000
نوع الوقود      0.000000
dtype: float64

In [8]:
# description of the data
dataset.describe()

,قوة الماتور,price
count,6864.000000,6864.000000
mean,1740.494755,66933.643211
std,589.470714,57022.973482
min,106.000000,1111.000000
25%,1400.000000,36000.000000
50%,1600.000000,55000.000000
75%,2000.000000,83000.000000
max,9000.000000,999999.000000


In [9]:
categorical_data = dataset.select_dtypes(['object']).copy() 
categorical_data.describe()

,لون السيارة,نوع الوقود,أصل السيارة,رخصة السيارة,نوع الجير,الزجاج,عداد السيارة,عدد الركاب,وسيلة الدفع,معروضة,أصحاب سابقون,إضافات
count,6864,6864,6864,6864,6864,6864,4923,6087,6864,6864,5214,6864
unique,26,4,6,2,3,2,1261,43,2,3,360,196
top,أبيض,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,100000,4+1,نقدا فقط,للبيع فقط,يد اولى,\n\nمُكيّف\nإغلاق مركزي\nجهاز إنذار\nمسجل CD\n...
freq,1446,3970,5833,6789,3623,6338,107,4847,4323,4209,505,1254


###### Observe that numeric features stored as string.... need to be handled

In [10]:
for feature in dataset:
    print(f'Feature: {feature}\n{dataset[feature].nunique()}: {dataset[feature].unique()}\n')

Feature: لون السيارة
26: ['أبيض عاجي ' 'سكني ' 'بيج ' 'فضي ' 'أسود ' 'بترولي ' 'أبيض '
 'أسود ميتالك ' 'ذهبي ' 'فيراني ' 'أحمر ' 'أزرق ' 'أزرق سماوي ' 'رمادي '
 'عدة الوان ' 'رصاصي ' 'برتقالي ' 'أخضر ' 'بني ' 'خمري ' 'كحلي ' 'زيتي '
 'بنفسجي ' 'جيشي ' 'أصفر ' 'كرميدي ']

Feature: نوع الوقود
4: ['بنزين ' 'ديزل ' 'هايبرد ' 'كهرباء ']

Feature: أصل السيارة
6: ['خصوصي ' 'عمومي ' 'تأجير ' 'تجاري ' 'تدريب سياقة ' 'حكومي ']

Feature: رخصة السيارة
2: ['فلسطينية ' 'نمرة صفراء ']

Feature: نوع الجير
3: ['اوتوماتيك ' 'نصف اوتوماتيك ' 'عادي ']

Feature: الزجاج
2: ['الكتروني ' 'يدوي ']

Feature: قوة الماتور
144: [2000 2500 1600  906 1200 1400 4500 1800 2200 1250 1300 1500 1100 3700
 1235 1900 2400 1000  125  800  220  420 1999 3600 3000 1850 4800  200
 3100 2300 3500 1150 3200 4000  300 2800  390 1995  900 1990 2008 2700
 5985 2900  903 1595 7000  250  270 6500 1499  550 2600 1950 1350  240
 3222 1690  170 1009 9000  230  905 3300  460 2996 5983  160  260 1700
  340 1234  180 5800 1190 1498 4750  2

In [11]:
dict_features_unique_values = {}
for feature in dataset:
    dict_features_unique_values[feature] = (dataset[feature].nunique(), dataset[feature].unique())
    
dict_features_unique_values

{'لون السيارة': (26,
  array(['أبيض عاجي ', 'سكني ', 'بيج ', 'فضي ', 'أسود ', 'بترولي ', 'أبيض ',
         'أسود ميتالك ', 'ذهبي ', 'فيراني ', 'أحمر ', 'أزرق ',
         'أزرق سماوي ', 'رمادي ', 'عدة الوان ', 'رصاصي ', 'برتقالي ',
         'أخضر ', 'بني ', 'خمري ', 'كحلي ', 'زيتي ', 'بنفسجي ', 'جيشي ',
         'أصفر ', 'كرميدي '], dtype=object)),
 'نوع الوقود': (4,
  array(['بنزين ', 'ديزل ', 'هايبرد ', 'كهرباء '], dtype=object)),
 'أصل السيارة': (6,
  array(['خصوصي ', 'عمومي ', 'تأجير ', 'تجاري ', 'تدريب سياقة ', 'حكومي '],
        dtype=object)),
 'رخصة السيارة': (2, array(['فلسطينية ', 'نمرة صفراء '], dtype=object)),
 'نوع الجير': (3,
  array(['اوتوماتيك ', 'نصف اوتوماتيك ', 'عادي '], dtype=object)),
 'الزجاج': (2, array(['الكتروني ', 'يدوي '], dtype=object)),
 'قوة الماتور': (144,
  array([2000, 2500, 1600,  906, 1200, 1400, 4500, 1800, 2200, 1250, 1300,
         1500, 1100, 3700, 1235, 1900, 2400, 1000,  125,  800,  220,  420,
         1999, 3600, 3000, 1850, 4800,  200, 3100, 23

###### Observe having plenty inconsistent data in the features needed to be handled

### Handle inconsistent data in 'عدد الركاب' feature

In [12]:
# Observe number of samples with no '+' sign along with their values
no_sum, no_sum_indices, naan_indices = [], [], []
for indx, car_passengers in enumerate(dataset['عدد الركاب']):
    if car_passengers is np.nan:
        naan_indices.append(indx)
    elif '+' not in car_passengers and not car_passengers.strip().isdigit():
        no_sum_indices.append(indx)
        no_sum.append((indx, car_passengers))

In [13]:
print(no_sum_indices, end='')

[51, 157, 335, 391, 462, 576, 704, 1304, 1692, 4016, 4189, 4661, 5029, 5296, 5776, 5806]

In [14]:
print(len(no_sum))
print(no_sum, end='')

16
[(51, '7:1 '), (157, '6±١ '), (335, '7.1 '), (391, '7 =1 '), (462, '6x1     '), (576, '2/1 '), (704, '4\\١ '), (1304, 'اكثر من 10 '), (1692, 'اكثر من 10 '), (4016, 'اكثر من 10 '), (4189, 'اكثر من 10 '), (4661, 'اكثر من 10 '), (5029, 'اكثر من 10 '), (5296, 'اكثر من 10 '), (5776, 'اكثر من 10 '), (5806, 'اكثر من 10 ')]

In [15]:
len(naan_indices)

777

In [16]:
for indx, car_passengers in enumerate(dataset['عدد الركاب']):
    if (car_passengers is not np.nan):
        if ('+' in car_passengers):         # for elem in calculations_list: if elem in car_passengers: car_passengers_number = car_passengers.split(elem)
            car_passengers_number = car_passengers.split('+')
            sum = 0
            for val in car_passengers_number:
                val = val.strip(" ,")
                if val.isdigit():
                    sum += int(val)
            dataset['عدد الركاب'].iloc[indx] = sum

        elif car_passengers.strip().isdigit():
            dataset['عدد الركاب'].iloc[indx] = int(car_passengers.strip())

C:\Users\DELL\AppData\Local\Temp\ipykernel_15956\3962871073.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['عدد الركاب'].iloc[indx] = sum
C:\Users\DELL\AppData\Local\Temp\ipykernel_15956\3962871073.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['عدد الركاب'].iloc[indx] = int(car_passengers.strip())


In [17]:
# drop samples that is neither nan nor have '+' sign
dataset.drop(labels=no_sum_indices, axis=0, inplace=True)

In [18]:
# Observe the shape of the data to asure deleting 32 samples
dataset.shape

(6848, 14)

In [19]:
dataset['عدد الركاب'] = dataset['عدد الركاب'].astype(float)

In [20]:
dataset['عدد الركاب']

0       5.0
1       8.0
2       NaN
3       5.0
4       5.0
       ... 
6859    5.0
6860    8.0
6861    5.0
6862    7.0
6863    8.0
Name: عدد الركاب, Length: 6848, dtype: float64

### Handle inconsistent data in 'أصحاب سابقون' feature

In [21]:
(dataset['أصحاب سابقون'].nunique(), dataset['أصحاب سابقون'].unique())

(360,
 array(['يد اولى ', '2 ', nan, '00  ', '0 ', '1 ', 'يد ثانية ', '4 ',
        'يد ثانيه ', '3 ', 'ثالثة ', 'غير منمر استيراد الماني ', '00 ',
        'غير منمرة ', 'أولى ', '٨ ', 'يد ثالثة ', 'يد ثالثة  ', 'ثانية ',
        'يد أولى ', 'يد سادسة ', '5 ', 'اولى  ', 'صفر ', '9 ', '0000 ',
        'رابعه ', 'يد أولى  ', '٤ ', 'يد رابعة  ', '٢ ', 'يد ثالثه ', '6 ',
        'يد اولى  ', '7 ', '... ', 'يد ثانيه  ', 'يد أولى استيراد ',
        'يد اولي 1 ', 'اولى ', '...... ', 'ثانيه ', 'واحد ', 'اربعة ',
        'اولا ', '٣ ', 'غير منمره ', 'يد تالته ', '٦ ', 'يد رابعه ',
        'يد ثالثة ( فتيات) ', '159000 ', '12 ', '- ', 'مش منمر ',
        '0 مستورده ', 'يد تانيه ', '10 ', 'يد 1 ', 'مستورد ', 'يد 4 ',
        'يد رابعة ', '00000 ', '١ ', 'استيراد مباشر من كوريا  ',
        'يد خامسة ', '٥ ', 'يد 5 ', '******** ', 'نعم ', 'مش منمرة  ',
        'يد 3 ', ' يد ثانيه ', 'يداولى ', 'يد اولا ', '4  ',
        'يد أولى وكالة ', 'يد خامسه ', 'يد ثانية  ', '٩ ', '٧ ',
        'يدو اولى ', '

In [22]:
unique_dic_gathered = {new_list: [] for new_list in range(13)}
unique_dic_gathered[15] = []
unique_dic_gathered[17] = []
 
unique_previous_owners_drivers = list(dataset['أصحاب سابقون'].unique())
 
for val in unique_previous_owners_drivers:
    if isinstance(val, str):
        if '١٧' in val or '17' in val:  # var_17 = ['17', '١٧']   ............. if any(elem in val for elem in var_17)
            unique_dic_gathered[17].append(val)
        if '١٥' in val or '15' in val:
            unique_dic_gathered[15].append(val)
        if 'اتنا عشر' in val or 'اثنا عشر' in val or '12' in val or '١٢' in val:
            unique_dic_gathered[12].append(val)
        elif 'حداش' in val or 'احدى عشر' in val or '11' in val or 'إحدى عشر' in val or 'أحدى عشر' in val or '١١' in val:
            unique_dic_gathered[11].append(val)
        elif 'عشرة' in val or 'عشره' in val or '10' in val or '١٠' in val:
            unique_dic_gathered[10].append(val)
        elif 'تسعة' in val or 'تسعه' in val or '9' in val or '٩' in val:
            unique_dic_gathered[9].append(val)        
        elif 'تامنة' in val or 'تامنه' in val or '8' in val or 'ثامنة' in val or 'ثامنه' in val or '٨' in val:
            unique_dic_gathered[8].append(val)
        elif 'سابعة' in val or 'سابعه' in val or '7' in val or '٧' in val:
            unique_dic_gathered[7].append(val)
        elif 'سادس' in val or '6' in val or '٦' in val: # 'سادسة' in val or 'سادسه' in val
            unique_dic_gathered[6].append(val)
        elif 'خامسة' in val or 'خامسه' in val or '5' in val or '٥' in val:
            unique_dic_gathered[5].append(val)
        elif 'رابعة' in val or 'رابعه' in val or '4' in val or '٤' in val or 'اربعه' in val or 'اربعة' in val or 'أربعه' in val or 'أربعة' in val:
            unique_dic_gathered[4].append(val)
        elif 'ثالثة' in val or 'ثالثه' in val or '3' in val or 'ثالتة' in val or 'تالثة' in val or 'ثالته' in val or 'تالثه' in val or 'تالتة' in val or 'تالته' in val or '٣' in val:
            unique_dic_gathered[3].append(val)
        elif 'ثانيه' in val or 'ثانية' in val or 'تانيه' in val or 'تانية' in val or '2' in val or '٢' in val:
            unique_dic_gathered[2].append(val)
        elif 'واله' in val or 'اولة' in val or 'اوله' in val or 'اولي' in val or 'اولئ' in val or         'اولى' in val or 'أولى' in val or '1' in val or '1 ' in val or 'يد اولى ' in val or 'يد. اولى' in val or 'واحد ' in val or 'اولا ' in val or '١ ' in val:
            unique_dic_gathered[1].append(val)
        elif 'zero' in val or 'صفر' in val or '0' in val or '٠' in val or '..' in val:
            unique_dic_gathered[0].append(val)

In [23]:
pre_drivers_0 = ['00  ', '0 ', '00 ', 'صفر ', '0000 ', '00000 ', '0 مستورده ', '0مستورد ', 
       'يد صفر  ', 'يد صفر ', '0استيراد  ', '000000 ', 'اصحاب سابقون صفر ', '000 ', 
       'صفر  ', 'صفرض ', '0 مستورد كوري ', '00000000 ', 'يد أولئ أصحاب سابقون صفر ', 
       'يد 0 ', 'صفرر ', '، 0 ', ' 0 ', '0 استيراد كوري  ', '0000000 ', ' 00 ', 'مش منمر . (صفر) ', 
       '0 استيراد  ', '٠ ', '٠٠٠٠ ', '٠٠٠٠٠٠٠٠٠ ', '٠٠٠٠٠٠ ', '٠٠٠٠٠٠٠٠٠٠ ', '٠٠ ','٠٠٠٠٠٠٠٠ ' ,
       '٠٠٠٠٠ ', '٠٠٠٠٠٠٠ ', '... ', '...... ', '......... ', 'Zero ', 'لا يوجد ', '. ', 'لايوجد ', 
       '.... ', 'من الشركه مباشر لا يوجد اصحاب سابقون ', 'Ooo ', 'استيراد كوريا - جديد مش منمر ', 
       '  O ',  '.. ', 'لا شيء ', 'O '
       ]

pre_drivers_1 = ['1 ', 'يد اولى ', 1, 'يد. اولى', 'واحد ', 'اولا ', '١ ',
      'أولى ', 'يد أولى ', 'اولى  ', 'يد أولى  ', 'يد اولى  ', 'يد أولى استيراد ', 'يد اولي 1 ', 'اولى ', 'يد 1 ', 'يداولى ', 'يد أولى وكالة ', 'يدو اولى ', 'يد اولى شركة ', 
      'يد. اولى ', 'ياولى ', 'يد اولى وكالة ', 'يد اولى  اصل شركة ', 'يد اولى اصحاب سابقون 0 ', 'أولى  ', 'وارد كوريا يد اولى ', 'يد اولى صفر كيلو  ', 'مستورده يد اولى ', 
      'ايد اولى ', '1  ', 'يد أولى بعد الاستيراد ', 'يد أولى 1 ', 'ايد اولى  ', ' 1 ', 'يد اولى فقط ', 'يد أولى،... ', 'اولى\x89 ', 'فقط يد اولى ', 'يد اولى، استيراد شخصي ', 
      'يدو اولى  ', 'يد اولى ( شركة ) ', 'يدأولى ', 'يداولى سحب شركه ', 'يد اولى استيراد شخصي ', 'ايد أولى  ', '01 ', 'يد اولى الماني  ', 'يد اولىً ', 'يد اولى / استيراد ', 
      '1 أولى  ', 'مستوردة يد اولى ', 'بد اولى مستورد ', 'يد اولى من الشركة المتحدة ',
      'يد اولا ', 'يداولا ', 'يد اولا  ', 'يد ١ ', 'يد  ١ ', 'يد واحد  ', '11111 ', 'يد واله ', 'يد اولئ ', 'يد اولي', 'ايد اوله ', 'يد اولة  ', 
      'اوله  ', 'يد اول',  'يدى اوله  ', 'يد اولي  ', 'اول ', 'يد ولا ', 'يد اول  ', 'يد اولي ', 'اوله ', 'يد ولى ', 'يد أولي ', 'يد اول ', 
      'يد أولئ  ', 'يد ا ', 'يد اوله ', 'يد واحدة  ', 'يد ', 'يد اولة ',  '4 نفس الشخص ', 'يد اةلي '
      ]

pre_drivers_2 = ['2 ', 'يد ثانية ', 'يد ثانيه ', 'ثانية ', '٢ ', 'يد ثانيه  ', 'ثانيه ', 'يد تانيه ', ' يد ثانيه ', 
      'يد ثانية  ', '2  ', 'يد 2 ', 'يدثانيه  ', 'ايد ثانيه ', 'يد ٢ ', ' يد ثانية ', 'يدثانيه ', 'يد تانية ', 
      'ايد 2 ', ' 2 ', 'أيد تانية ', 'يدثانية ', 'تانيه ', '2يد  ', 'يدتانيه ', 'ثانية  ', 'ايد ثانية  ', '2يد ', 
      'يد٢ ', 'يد2 ', 'ايدثانيه ', 'يد  ثانية ', 'يد تانيه  ', 'تانية ', 'الثانية ', 'يد تانية  ', '@ ',
      'اثنان ', 'يد تاني ', 'يد ثني ', 'يد ثاتيه ', 'يد ثاني ', 'ثانبه ', 'يد ثاني.  ', 'تانبه ', 'يد ثانيا ', 
      'تاني  ', 'ثاني ', 'يديانيه ', 'يد ثاني  ',
      ]

pre_drivers_3 = ['3 ', 'ثالثة ', 'يد ثالثة ', 'يد ثالثة  ', 'يد ثالثه ', '٣ ', 'يد تالته ', 'يد ثالثة ( فتيات) ', 
       'يد 3 ', 'ثالثه ', '03 ', 'ثالثة  ', '13 ', 'يد ثالثه  ', 'يد3 ', 'يدثالثة ', 'عدد٣ ', 'تالته ', 
       'يد ٣ ', 'يدثالثه ', 'يد٣ ', 'يد التالته ', ' ايد ثالثه ', 'تالثه ', '3  ', ',3 ', 'ايد ثالثة  ', 'ايد 3 ',
       'يد ثالث ', 'ثالث ', 'ثالثى ',  'يد ثالثي ',
       ]

pre_drivers_4 = ['4 ', 'رابعه ', '٤ ', 'يد رابعة  ', 'اربعة ', 'يد رابعه ', 'يد 4 ', 'يد رابعة ', '4  ', 'رابعة ', 
       'يد4 ', 'يد ٤ بس نخب السيارة ', 'يد ٤ ', 'يد٤ ', 'يد رابعه  ', 'ايد رابعه ', '14 ', '04 ', 'يد اربعه ', 
       'رابعه  ', '٤٤٤٤٤ ']

pre_drivers_5 = ['5 ', 'يد خامسة ', '٥ ', 'يد 5 ', 'يد خامسه ', 'يد خامسة  ', ' 5 ', 'يدخامسه ', 'خامسة ', 
              'يد٥ ', 'خامسه ', 'يد ٥ ', '  5 ', 'يد خامساً  ', 'يد خمسة ', 'خمسه ', 'خمسة ', 'خمس ', ]

pre_drivers_6 = ['يد سادسة ', '6 ', '٦ ', 'يد6 ', '  6 ', 'يد 6 ']

pre_drivers_7 = ['7 ', '٧ ', 'يد سابعة ', 'يد سابعه ', 'يد ٧  ', 'سابعه ']

pre_drivers_8 = ['٨ ', '8 ', '١٨ ', 'يد ثامنه ', 'يد 8 ', 'ثمانيه ']

pre_drivers_9 = ['9 ', '٩ ', '٠٠٠٠٩ ', 'يد ٩  ', 'يد تاسعة ']

pre_drivers_10 = ['10 ', '١٠ ', 'يد عاشره ', 'يد عاشرة ']

pre_drivers_11 = ['11 ', '١١ ']


pre_drivers_12 = ['12 ', '١٢ ']
pre_drivers_15 = ['١٥ ', '15 ']
pre_drivers_16 = ['16 ']
pre_drivers_17 = ['17 ', 'يد ١٧ ', '١٧ ', '١٧', '١٧ ', '١٧ ', 'يد ١٧ ',
              '17 ', 'يد ١٧ ' , '١٧ ', 'يد ١٧ ', '١٧ '
                  ]
pre_drivers_18 = ['18 ']
pre_drivers_19 = ['19 ']

pre_drivers_importation = ['مستورد شركة  ', 'استيراد الماني ', ' استيراد الماني ', 'مستورد ', 'استيراد مباشر من كوريا  ', 
                            'استيراد كوري ', 'مستوردة ', 'استيراد ', 'استيراد كوريا ', 'استيراد مباشر من اوروبا  ', 
                            'وارد المانيا ', 'استراد  ', 'استيراد المانيا ', 'استيراد كوريا  ', 'مستورده  ', 'مستوردة  ',  'أستيراد ألماني  ', 
                            'مستوردة غير مرخصة ', 'استراد شخصي ',
                            'مستورد  ', 'مستورده ', 'استيراد مباشر ', 
                            ]

car_no_number = ['غير منمر استيراد الماني ', 'غير منمرة ', 'غير منمره ', 'مش منمر ', 'مش منمرة  ', 'مش منمرة ', 
              'غير منمر  ', 'مستود غير منمر ', 'مش منمرا ', 'غير منمرة  ', 'غير منمر ', 'مش منمر  ', 'غير منمرمستوردة ',
              'مستوردة غير منمرة ', 'مش منمره '
              ]
pre_drivers_yes_no_number = ['نعم ', ' منمره  ']

null_values = ['- ', '******** ', 'ا ', '********** ', '****** ', '--- ', '؟ ', '? ', 'E ', '، ', 
              'للبيع  ', 'اصل تدريب سواقه  ', 'انا ', 'سحب شركة ', 'شطب ', 'اصل شركة ', 'مشطوب ', 'خصوصي  '
              ]
large_num = ['159000 ', '56 ', '170000 ', '78888877766 ', '0778244 ', '1000 ', '123423 ', '12345 ', 'اشخاص ']

In [24]:
pre_drivers = [pre_drivers_0, pre_drivers_1, pre_drivers_2, pre_drivers_3, pre_drivers_4, 
pre_drivers_5, pre_drivers_6, pre_drivers_7, pre_drivers_8,pre_drivers_9, 
pre_drivers_10, pre_drivers_11, pre_drivers_12, pre_drivers_15, pre_drivers_16, 
pre_drivers_17, pre_drivers_18, pre_drivers_19, null_values, car_no_number,
pre_drivers_yes_no_number, large_num, pre_drivers_importation]

In [25]:
dict_pre_drivers = {
  **dict.fromkeys(pre_drivers_0, 0), 
  **dict.fromkeys(car_no_number, 0), 
  **dict.fromkeys(pre_drivers_yes_no_number, 0), 
  **dict.fromkeys(pre_drivers_importation, 0),
  **dict.fromkeys(pre_drivers_1, 1), 
  **dict.fromkeys(pre_drivers_2, 2), 
  **dict.fromkeys(pre_drivers_3, 3), 
  **dict.fromkeys(pre_drivers_4, 4), 
  **dict.fromkeys(pre_drivers_5, 5), 
  **dict.fromkeys(pre_drivers_6, 6), 
  **dict.fromkeys(pre_drivers_7, 7), 
  **dict.fromkeys(pre_drivers_8, 8), 
  **dict.fromkeys(pre_drivers_9, 9), 
  **dict.fromkeys(pre_drivers_10, 10), 
  **dict.fromkeys(pre_drivers_11, 11), 
  **dict.fromkeys(pre_drivers_12, 12), 
  **dict.fromkeys(pre_drivers_15, 15), 
  **dict.fromkeys(pre_drivers_16, 16),
  **dict.fromkeys(pre_drivers_17, 17),
  **dict.fromkeys(pre_drivers_18, 18), 
  **dict.fromkeys(pre_drivers_19, 19),
  **dict.fromkeys(null_values, np.NaN), 
  **dict.fromkeys(large_num, np.NaN), 
  # **dict.fromkeys((np.NaN), None),
}

In [26]:
dataset['أصحاب سابقون'].replace(dict_pre_drivers, inplace=True)

In [27]:
print(f"{dataset['أصحاب سابقون'].nunique()} => {dataset['أصحاب سابقون'].unique()}")

18 => [ 1.  2. nan  0.  4.  3.  8.  6.  5.  9.  7. 12. 10. 11. 16. 17. 15. 18.
 19.]


In [28]:
dataset['أصحاب سابقون']

0       1.0
1       2.0
2       NaN
3       0.0
4       NaN
       ... 
6859    2.0
6860    5.0
6861    0.0
6862    2.0
6863    5.0
Name: أصحاب سابقون, Length: 6848, dtype: float64

### Handle inconsistent data in 'عداد السيارة' feature

In [29]:
print(f"Feature: 'عداد السيارة']\n{dataset['عداد السيارة'].nunique()}: {dataset['عداد السيارة'].unique()}\n")

Feature: 'عداد السيارة']
1259: ['75000 ' '130000 ' nan ... '222222 ' '177 ' '7686866 ']



In [30]:
dataset['عداد السيارة'][0]

'75000 '

In [266]:
# dataset['عداد السيارة'] = dataset['عداد السيارة'].str.strip() #.value_counts().plot.barh() 

In [31]:
dataset['عداد السيارة']

0         75000 
1        130000 
2            NaN
3          شغال 
4         38000 
          ...   
6859     130000 
6860      00000 
6861      50000 
6862        100 
6863    7686866 
Name: عداد السيارة, Length: 6848, dtype: object

In [415]:
data_set_before_habed = dataset.copy()
data_set_before_habed

,لون السيارة,نوع الوقود,أصل السيارة,رخصة السيارة,نوع الجير,الزجاج,قوة الماتور,عداد السيارة,عدد الركاب,وسيلة الدفع,معروضة,أصحاب سابقون,إضافات,price
0,أبيض عاجي,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,2000,75000,8.000000,نقدا فقط,للبيع فقط,1.000000,\n\nمُكيّف\nإغلاق مركزي\nجهاز إنذار\nمسجل CD\n...,100000
1,سكني,ديزل,خصوصي,فلسطينية,نصف اوتوماتيك,الكتروني,2500,130000,7.000000,إمكانية التقسيط,للبيع أو التبديل,2.000000,\n\nمُكيّف\nإغلاق مركزي\nجهاز إنذار\nمسجل CD\n...,60000
2,سكني,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1600,NaN,5.260841,نقدا فقط,للبيع فقط,2.171429,\n\nمُكيّف\nإغلاق مركزي\nجهاز إنذار\nمسجل CD\n...,43500
4,فضي,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1200,38000,7.000000,نقدا فقط,للبيع فقط,2.333333,\n\nمُكيّف\nإغلاق مركزي\nجهاز إنذار\nجنطات مغن...,54000
5,أسود,ديزل,خصوصي,فلسطينية,اوتوماتيك,الكتروني,2000,NaN,3.000000,نقدا فقط,للبيع فقط,0.000000,\n\nمُكيّف\nإغلاق مركزي\nجهاز إنذار\nمسجل CD\n...,205000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6859,فيراني,كهرباء,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1000,130000,5.000000,إمكانية التقسيط,للبيع فقط,2.000000,\n\nمُكيّف\nإغلاق مركزي\nجهاز إنذار\nجنطات مغن...,45000
6860,أبيض,ديزل,عمومي,فلسطينية,عادي,الكتروني,2400,0,8.000000,نقدا فقط,للبيع فقط,5.000000,\n\n\n,48000
6861,فضي,ديزل,خصوصي,فلسطينية,عادي,الكتروني,1600,50000,5.000000,نقدا فقط,للبيع فقط,0.000000,\n\nمُكيّف\nإغلاق مركزي\nجهاز إنذار\nمسجل CD\n...,87000
6862,بترولي,ديزل,خصوصي,فلسطينية,اوتوماتيك,الكتروني,2200,100,7.000000,نقدا فقط,للبيع فقط,2.000000,\n\nمُكيّف\nإغلاق مركزي\nجهاز إنذار\nمسجل CD\n...,126000


In [33]:
to_be_deleted = ['_', '-', '!!!!', '_', '...', 'شغال', 'واحد',
                'كيلو', 'قابل', 'كيلو', ' ', 'واحد']

In [34]:
types = []

In [35]:
copyyy = dataset.copy()

In [36]:
indices_values_to_delete = []
for indx, sample in enumerate(dataset['عداد السيارة']):
    if sample is not np.nan:
        if sample.strip().isdigit():
            dataset['عداد السيارة'].iloc[indx] = int(sample.strip())   # dataset['عداد السيارة'].replace('الف', 000)
        else:
            extract_num = re.sub(r'[^0-9]', '', sample)
            if extract_num.isdigit():
                dataset['عداد السيارة'].iloc[indx] = int(extract_num)

C:\Users\DELL\AppData\Local\Temp\ipykernel_15956\1182205757.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['عداد السيارة'].iloc[indx] = int(sample.strip())   # dataset['عداد السيارة'].replace('الف', 000)
C:\Users\DELL\AppData\Local\Temp\ipykernel_15956\1182205757.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['عداد السيارة'].iloc[indx] = int(extract_num)


3 => شغال 
|||شغال |||
17 => 150 الف  
|||150|||
28 => 122,000 
|||122000|||
56 => _ 
|||_ |||
60 => - 
|||- |||
83 => 80 الف 
|||80|||
88 => 120000قابل لزيادة  
|||120000|||
100 => 120 الف 
|||120|||
180 => 60الف 
|||60|||
206 => 47 الف 
|||47|||
225 => 90000 كيلو 
|||90000|||
227 => 000 كيلو متر 
|||0|||
252 => شغال 
|||شغال |||
269 => 140 الف كيلو متر  
|||140|||
295 => ..... 
|||..... |||
302 => ١٩٠,٠٠٠ 
|||١٩٠,٠٠٠ |||
306 => 16000 كم 
|||16000|||
307 => .... 
|||.... |||
322 => واحد 
|||واحد |||
329 => شغال 
|||شغال |||
339 => 5500 كيلومتر 
|||5500|||
357 => 40 الف 
|||40|||
405 => 250.000 
|||250000|||
417 => 80الف 
|||80|||
433 => ................ 
|||................ |||
440 => شغال 
|||شغال |||
465 => ******** 
|||******** |||
487 => 75000كيلو متر 
|||75000|||
502 => 91 الف 
|||91|||
566 => 180 كيلومتر 
|||180|||
579 => 5400  كيلو 
|||5400|||
580 => 18كم 
|||18|||
588 => 150 الف 
|||150|||
633 => شغال 
|||شغال |||
637 => شغال 
|||شغال |||
645 => 105000كيلو متر  
|||105000|||
6

In [38]:
print(type(dataset['عداد السيارة'][1]))
dataset['عداد السيارة'][226]

<class 'int'>


nan

In [39]:
dataset['عداد السيارة'].shape

(6848,)

## Handle missing data

In [45]:
model = KNNImputer(n_neighbors=3)           
after_imputation = model.fit_transform(dataset[['أصحاب سابقون', 'عدد الركاب']])
after_imputation

array([[1.        , 5.        ],
       [2.        , 8.        ],
       [2.1696325 , 5.26008895],
       ...,
       [0.        , 5.        ],
       [2.        , 7.        ],
       [5.        , 8.        ]])

In [46]:
dataset['أصحاب سابقون'] = after_imputation[:,0]
dataset['عدد الركاب'] = after_imputation[:,1]

In [47]:
dataset

,لون السيارة,نوع الوقود,أصل السيارة,رخصة السيارة,نوع الجير,الزجاج,قوة الماتور,عداد السيارة,عدد الركاب,وسيلة الدفع,معروضة,أصحاب سابقون,إضافات,price
0,أبيض عاجي,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,2000,75000,5.000000,نقدا فقط,للبيع فقط,1.000000,\n\nمُكيّف\nإغلاق مركزي\nجهاز إنذار\nمسجل CD\n...,100000
1,سكني,ديزل,خصوصي,فلسطينية,نصف اوتوماتيك,الكتروني,2500,130000,8.000000,إمكانية التقسيط,للبيع أو التبديل,2.000000,\n\nمُكيّف\nإغلاق مركزي\nجهاز إنذار\nمسجل CD\n...,60000
2,سكني,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1600,NaN,5.260089,نقدا فقط,للبيع فقط,2.169632,\n\nمُكيّف\nإغلاق مركزي\nجهاز إنذار\nمسجل CD\n...,43500
3,بيج,بنزين,خصوصي,فلسطينية,عادي,يدوي,906,شغال,5.000000,إمكانية التقسيط,للبيع فقط,0.000000,\n\nمُكيّف\nمسجل CD\n\n,5500
4,فضي,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1200,38000,5.000000,نقدا فقط,للبيع فقط,0.666667,\n\nمُكيّف\nإغلاق مركزي\nجهاز إنذار\nجنطات مغن...,54000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6859,فيراني,كهرباء,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1000,130000,5.000000,إمكانية التقسيط,للبيع فقط,2.000000,\n\nمُكيّف\nإغلاق مركزي\nجهاز إنذار\nجنطات مغن...,45000
6860,أبيض,ديزل,عمومي,فلسطينية,عادي,الكتروني,2400,0,8.000000,نقدا فقط,للبيع فقط,5.000000,\n\n\n,48000
6861,فضي,ديزل,خصوصي,فلسطينية,عادي,الكتروني,1600,50000,5.000000,نقدا فقط,للبيع فقط,0.000000,\n\nمُكيّف\nإغلاق مركزي\nجهاز إنذار\nمسجل CD\n...,87000
6862,بترولي,ديزل,خصوصي,فلسطينية,اوتوماتيك,الكتروني,2200,100,7.000000,نقدا فقط,للبيع فقط,2.000000,\n\nمُكيّف\nإغلاق مركزي\nجهاز إنذار\nمسجل CD\n...,126000


### Feature Engineering

In [48]:
# dataset.drop('إضافات', inplace=True, axis=1) 
data_drop_cols = dataset.drop(['إضافات', 'عداد السيارة'], axis=1) 

In [49]:
data_drop_cols

,لون السيارة,نوع الوقود,أصل السيارة,رخصة السيارة,نوع الجير,الزجاج,قوة الماتور,عدد الركاب,وسيلة الدفع,معروضة,أصحاب سابقون,price
0,أبيض عاجي,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,2000,5.000000,نقدا فقط,للبيع فقط,1.000000,100000
1,سكني,ديزل,خصوصي,فلسطينية,نصف اوتوماتيك,الكتروني,2500,8.000000,إمكانية التقسيط,للبيع أو التبديل,2.000000,60000
2,سكني,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1600,5.260089,نقدا فقط,للبيع فقط,2.169632,43500
3,بيج,بنزين,خصوصي,فلسطينية,عادي,يدوي,906,5.000000,إمكانية التقسيط,للبيع فقط,0.000000,5500
4,فضي,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1200,5.000000,نقدا فقط,للبيع فقط,0.666667,54000
...,...,...,...,...,...,...,...,...,...,...,...,...
6859,فيراني,كهرباء,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1000,5.000000,إمكانية التقسيط,للبيع فقط,2.000000,45000
6860,أبيض,ديزل,عمومي,فلسطينية,عادي,الكتروني,2400,8.000000,نقدا فقط,للبيع فقط,5.000000,48000
6861,فضي,ديزل,خصوصي,فلسطينية,عادي,الكتروني,1600,5.000000,نقدا فقط,للبيع فقط,0.000000,87000
6862,بترولي,ديزل,خصوصي,فلسطينية,اوتوماتيك,الكتروني,2200,7.000000,نقدا فقط,للبيع فقط,2.000000,126000


In [50]:
# Observe null values proportion for each feature
dataset.isnull().mean().sort_values(ascending=False).head()

عداد السيارة    0.282272
لون السيارة     0.000000
نوع الوقود      0.000000
أصل السيارة     0.000000
رخصة السيارة    0.000000
dtype: float64

### Encode the data

In [51]:
categorical_features = ['لون السيارة', 'نوع الوقود', 'أصل السيارة', 'رخصة السيارة', 'نوع الجير', 'الزجاج', 'وسيلة الدفع', 'معروضة']
categorical_data = data_drop_cols[categorical_features]

In [52]:
categorical_data

,لون السيارة,نوع الوقود,أصل السيارة,رخصة السيارة,نوع الجير,الزجاج,وسيلة الدفع,معروضة
0,أبيض عاجي,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,نقدا فقط,للبيع فقط
1,سكني,ديزل,خصوصي,فلسطينية,نصف اوتوماتيك,الكتروني,إمكانية التقسيط,للبيع أو التبديل
2,سكني,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,نقدا فقط,للبيع فقط
3,بيج,بنزين,خصوصي,فلسطينية,عادي,يدوي,إمكانية التقسيط,للبيع فقط
4,فضي,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,نقدا فقط,للبيع فقط
...,...,...,...,...,...,...,...,...
6859,فيراني,كهرباء,خصوصي,فلسطينية,اوتوماتيك,الكتروني,إمكانية التقسيط,للبيع فقط
6860,أبيض,ديزل,عمومي,فلسطينية,عادي,الكتروني,نقدا فقط,للبيع فقط
6861,فضي,ديزل,خصوصي,فلسطينية,عادي,الكتروني,نقدا فقط,للبيع فقط
6862,بترولي,ديزل,خصوصي,فلسطينية,اوتوماتيك,الكتروني,نقدا فقط,للبيع فقط


In [53]:
ohe = OHE2()
ohe_data = ohe.fit_transform(categorical_data)

C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\category_encoders\one_hot.py:145: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for cat_name, class_ in values.iteritems():
C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\category_encoders\one_hot.py:145: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for cat_name, class_ in values.iteritems():
C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\category_encoders\one_hot.py:145: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for cat_name, class_ in values.iteritems():
C:\Users\DELL\AppData\Local\Packages\PythonSof

In [54]:
ohe_data

,لون السيارة_1,لون السيارة_2,لون السيارة_3,لون السيارة_4,لون السيارة_5,لون السيارة_6,لون السيارة_7,لون السيارة_8,لون السيارة_9,لون السيارة_10,...,نوع الجير_1,نوع الجير_2,نوع الجير_3,الزجاج_1,الزجاج_2,وسيلة الدفع_1,وسيلة الدفع_2,معروضة_1,معروضة_2,معروضة_3
0,1,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,1,0,1,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,1,0,1,0
2,0,1,0,0,0,0,0,0,0,0,...,1,0,0,1,0,1,0,1,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,1,0,1,1,0,0
4,0,0,0,1,0,0,0,0,0,0,...,1,0,0,1,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6859,0,0,0,0,0,0,0,0,0,1,...,1,0,0,1,0,0,1,1,0,0
6860,0,0,0,0,0,0,1,0,0,0,...,0,0,1,1,0,1,0,1,0,0
6861,0,0,0,1,0,0,0,0,0,0,...,0,0,1,1,0,1,0,1,0,0
6862,0,0,0,0,0,1,0,0,0,0,...,1,0,0,1,0,1,0,1,0,0


In [55]:
data_drop_cols = pd.concat([data_drop_cols,ohe_data], axis=1)

In [56]:
data_drop_cols

,لون السيارة,نوع الوقود,أصل السيارة,رخصة السيارة,نوع الجير,الزجاج,قوة الماتور,عدد الركاب,وسيلة الدفع,معروضة,...,نوع الجير_1,نوع الجير_2,نوع الجير_3,الزجاج_1,الزجاج_2,وسيلة الدفع_1,وسيلة الدفع_2,معروضة_1,معروضة_2,معروضة_3
0,أبيض عاجي,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,2000,5.000000,نقدا فقط,للبيع فقط,...,1,0,0,1,0,1,0,1,0,0
1,سكني,ديزل,خصوصي,فلسطينية,نصف اوتوماتيك,الكتروني,2500,8.000000,إمكانية التقسيط,للبيع أو التبديل,...,0,1,0,1,0,0,1,0,1,0
2,سكني,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1600,5.260089,نقدا فقط,للبيع فقط,...,1,0,0,1,0,1,0,1,0,0
3,بيج,بنزين,خصوصي,فلسطينية,عادي,يدوي,906,5.000000,إمكانية التقسيط,للبيع فقط,...,0,0,1,0,1,0,1,1,0,0
4,فضي,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1200,5.000000,نقدا فقط,للبيع فقط,...,1,0,0,1,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6859,فيراني,كهرباء,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1000,5.000000,إمكانية التقسيط,للبيع فقط,...,1,0,0,1,0,0,1,1,0,0
6860,أبيض,ديزل,عمومي,فلسطينية,عادي,الكتروني,2400,8.000000,نقدا فقط,للبيع فقط,...,0,0,1,1,0,1,0,1,0,0
6861,فضي,ديزل,خصوصي,فلسطينية,عادي,الكتروني,1600,5.000000,نقدا فقط,للبيع فقط,...,0,0,1,1,0,1,0,1,0,0
6862,بترولي,ديزل,خصوصي,فلسطينية,اوتوماتيك,الكتروني,2200,7.000000,نقدا فقط,للبيع فقط,...,1,0,0,1,0,1,0,1,0,0


In [57]:
data_drop_cols.drop(categorical_features, axis=1, inplace=True)